# Описание проекта

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# План проекта

1. Загрузить и подготовить данные.
2. Обучить разные модели с различными гиперпараметрами.
3. Проанализировать скорость работы и качество моделей.

Примечания:

    - Для оценки качества моделей применять метрику RMSE.
    - Самостоятельно освоить библиотеку LightGBM и её средствами построить модели градиентного бустинга.
    - Время выполнения ячейки кода Jupyter Notebook можно получить специальной командой. Найти её.
    - Поскольку модель градиентного бустинга может обучаться долго, изменять у неё только два-три параметра.
    - Если перестанет работать Jupyter Notebook, удалите лишние переменные оператором del: "del features_train"

# Описание данных

Признаки:

    - DateCrawled — дата скачивания анкеты из базы
    - VehicleType — тип автомобильного кузова
    - RegistrationYear — год регистрации автомобиля
    - Gearbox — тип коробки передач
    - Power — мощность (л. с.)
    - Model — модель автомобиля
    - Kilometer — пробег (км)
    - RegistrationMonth — месяц регистрации автомобиля
    - FuelType — тип топлива
    - Brand — марка автомобиля
    - NotRepaired — была машина в ремонте или нет
    - DateCreated — дата создания анкеты
    - NumberOfPictures — количество фотографий автомобиля
    - PostalCode — почтовый индекс владельца анкеты (пользователя)
    - LastSeen — дата последней активности пользователя

Целевой признак:
    
    - Price — цена (евро)

# 1. Подготовка данных

Загрузим все необходимые библиотеки для анализа.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from numpy.linalg import inv
from sklearn.utils import shuffle
from IPython.display import display
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from pymystem3 import Mystem
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
import time




Загрузим массив.

In [2]:
df = pd.read_csv('/datasets/autos.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


354369 строк, 15 столбцов. В качестве признаков нам не потребуются столбцы DateCrawled, DateCreated, NumberOfPictures, PostalCode и LastSeen. Уберем эти столбцы. Также, в столбцах VehicleType, Gearbox, Model, FuelType и NotRepaired есть пропуски - необходимо их изучить. Начнем с VehicleType (тип автомобильного кузова).

In [3]:
df = df.drop(columns = ['DateCrawled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 11 columns):
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
dtypes: int64(5), object(6)
memory usage: 29.7+ MB


In [4]:
df['VehicleType'].value_counts()

sedan          91457
small          79831
wagon          65166
bus            28775
convertible    20203
coupe          16163
suv            11996
other           3288
Name: VehicleType, dtype: int64

Не такое большое разнообразие типов.

Наиболее простым и быстрым способом будет удалить строки с пропусками. Тем более, что максимально достоверно заполнить пропуски у нас вряд ли получится.

In [5]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 245814 entries, 3 to 354367
Data columns (total 11 columns):
Price                245814 non-null int64
VehicleType          245814 non-null object
RegistrationYear     245814 non-null int64
Gearbox              245814 non-null object
Power                245814 non-null int64
Model                245814 non-null object
Kilometer            245814 non-null int64
RegistrationMonth    245814 non-null int64
FuelType             245814 non-null object
Brand                245814 non-null object
NotRepaired          245814 non-null object
dtypes: int64(5), object(6)
memory usage: 22.5+ MB


Изучим аномальные значения.

In [6]:
df['Price'].describe()

count    245814.000000
mean       5125.346717
std        4717.948673
min           0.000000
25%        1499.000000
50%        3500.000000
75%        7500.000000
max       20000.000000
Name: Price, dtype: float64

Максимальная цена - 20000 евро - ок, минимальная - ноль - сомнительно. Вряд ли бы кто-то решил просто так отдать транспорт. Тем более, что обычно на сайтах по продаже автомобилей стоит фильтр при вводе информации. Посмотрим на эти "сделки века", заодно включив туда цену меньше 5 евро(как мне кажется это порог нормальности).

In [7]:
df.query('Price <= 5')

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired
7,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no
89,1,sedan,1995,manual,113,e_klasse,150000,4,gasoline,mercedes_benz,no
152,0,bus,2004,manual,101,meriva,150000,10,lpg,opel,yes
268,1,sedan,1990,manual,90,80,70000,6,petrol,audi,no
579,0,sedan,1996,manual,170,5er,150000,0,petrol,bmw,no
...,...,...,...,...,...,...,...,...,...,...,...
353882,0,small,1996,manual,45,polo,150000,8,petrol,volkswagen,yes
353943,0,sedan,1999,manual,150,golf,125000,10,petrol,volkswagen,no
353995,0,sedan,1991,manual,133,100,150000,6,petrol,audi,no
354124,0,small,2004,manual,200,golf,150000,0,petrol,volkswagen,no


3737 автомобилей за бесплатно. Все разные. Вряд ли получится адекватно откорректировать или заново заполнить ключевой признак - удалим аномальные строчки.

In [8]:
df = df.query('Price > 5')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242077 entries, 3 to 354367
Data columns (total 11 columns):
Price                242077 non-null int64
VehicleType          242077 non-null object
RegistrationYear     242077 non-null int64
Gearbox              242077 non-null object
Power                242077 non-null int64
Model                242077 non-null object
Kilometer            242077 non-null int64
RegistrationMonth    242077 non-null int64
FuelType             242077 non-null object
Brand                242077 non-null object
NotRepaired          242077 non-null object
dtypes: int64(5), object(6)
memory usage: 22.2+ MB


In [9]:
df['VehicleType'].value_counts()

sedan          71142
small          57986
wagon          50367
bus            23406
convertible    16062
coupe          11927
suv             9538
other           1649
Name: VehicleType, dtype: int64

Все в порядке.

In [10]:
df['RegistrationYear'].describe()

count    242077.000000
mean       2002.978709
std           6.130774
min        1923.000000
25%        1999.000000
50%        2003.000000
75%        2007.000000
max        2018.000000
Name: RegistrationYear, dtype: float64

Самый старый автомобиль - 1923 года выпуска. Учитывая, что максимальная стоимость была 20000 евро - сомнительно. Посмотрим на эту строчку и заодно на все старше 1950 года.

In [11]:
df.query('RegistrationYear <= 1950')

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired
1928,7000,suv,1945,manual,48,other,150000,2,petrol,volkswagen,no
33638,5400,small,1937,manual,0,other,20000,3,petrol,opel,no
34055,2900,small,1937,manual,34,other,40000,7,petrol,ford,yes
60844,160,sedan,1950,manual,0,other,150000,2,petrol,mercedes_benz,no
68185,19900,convertible,1932,manual,18,other,5000,2,petrol,bmw,no
74942,12000,suv,1945,manual,45,other,50000,2,petrol,volkswagen,no
88033,6999,suv,1942,manual,72,other,150000,2,gasoline,jeep,no
103192,10000,coupe,1950,manual,130,other,5000,1,petrol,alfa_romeo,no
117371,8750,convertible,1923,manual,11,c3,5000,0,petrol,citroen,no
144046,15999,suv,1943,manual,54,other,50000,7,gasoline,jeep,no


Такая машина не одна, и, не зная состояния каждой конкретной машины, нам сложно судить о корректности того или иного объявления. Оставим их.

In [12]:
df['Gearbox'].value_counts()

manual    191680
auto       50397
Name: Gearbox, dtype: int64

С коробками передач без неожиданностей.

In [13]:
df['Power'].describe()

count    242077.000000
mean        120.128620
std         137.032613
min           0.000000
25%          75.000000
50%         111.000000
75%         150.000000
max       20000.000000
Name: Power, dtype: float64

Здесь есть вопросы - ноль и двадцать тысяч лошадиных сил? Ползком и на ракете. Посмотрим на них.

In [14]:
df.query('Power < 10 or Power > 1000')

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired
55,550,wagon,1999,manual,0,astra,150000,8,gasoline,opel,yes
70,800,small,1993,manual,0,polo,150000,3,petrol,volkswagen,no
98,4290,bus,2008,manual,0,combo,150000,2,gasoline,opel,no
158,800,sedan,1993,manual,0,golf,10000,9,petrol,volkswagen,yes
166,300,wagon,1998,manual,0,v40,150000,6,petrol,volvo,no
...,...,...,...,...,...,...,...,...,...,...,...
354263,1800,coupe,2000,manual,0,clk,150000,10,petrol,mercedes_benz,no
354332,7900,bus,2007,manual,0,b_klasse,125000,1,petrol,mercedes_benz,no
354335,390,small,1997,auto,0,corsa,100000,6,petrol,opel,yes
354360,3999,wagon,2005,manual,3,3er,150000,5,gasoline,bmw,no


9254 автомобиля или редчайших случаев исключительной технической мысли. Отбросим их.

In [15]:
df = df.query('Power > 10 and Power < 1000')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 232818 entries, 3 to 354367
Data columns (total 11 columns):
Price                232818 non-null int64
VehicleType          232818 non-null object
RegistrationYear     232818 non-null int64
Gearbox              232818 non-null object
Power                232818 non-null int64
Model                232818 non-null object
Kilometer            232818 non-null int64
RegistrationMonth    232818 non-null int64
FuelType             232818 non-null object
Brand                232818 non-null object
NotRepaired          232818 non-null object
dtypes: int64(5), object(6)
memory usage: 21.3+ MB


Столбец Model и Brand невозможно оценить на аномалии. Только на корректность внесения данных. Обычно при публикации объявления нам предлагается выбор из списка, а не свободный ввод значений марки и модели.

In [16]:
df['Brand'].value_counts()

volkswagen       49398
bmw              26163
opel             24443
mercedes_benz    22635
audi             20745
ford             16009
renault          10635
peugeot           7430
fiat              5912
seat              4683
skoda             4261
mazda             3766
toyota            3456
citroen           3444
nissan            3240
smart             3194
mini              2639
hyundai           2618
volvo             2368
mitsubishi        1910
honda             1831
kia               1760
alfa_romeo        1596
suzuki            1595
chevrolet         1196
chrysler           931
dacia              690
porsche            526
subaru             504
jeep               478
daihatsu           473
saab               401
land_rover         395
jaguar             367
daewoo             302
lancia             291
rover              239
trabant            166
lada               128
Name: Brand, dtype: int64

Есть даже trabant и нет категории other - выбросы с которыми ничего не надо делать.

In [17]:
df['Model'].value_counts()

golf                  19357
other                 17181
3er                   14226
polo                   8231
corsa                  7654
                      ...  
i3                        4
serie_3                   3
samara                    2
range_rover_evoque        2
rangerover                1
Name: Model, Length: 249, dtype: int64

249 значений. Выглядит удовлетворительно.

In [18]:
df['Kilometer'].describe()

count    232818.000000
mean     127017.004699
std       37063.426458
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: Kilometer, dtype: float64

Минимальный километраж - 5000, максимальный - 150000. Все в пределах обыденного.

In [19]:
df['RegistrationMonth'].value_counts()

3     25341
6     22316
4     20996
5     20634
7     19270
10    18773
9     17323
11    17280
12    17229
1     16553
8     16062
2     15495
0      5546
Name: RegistrationMonth, dtype: int64

Нулевой месяц - это какой? Придется и от этого избавиться.

In [20]:
df = df.query('RegistrationMonth != 0')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227272 entries, 3 to 354367
Data columns (total 11 columns):
Price                227272 non-null int64
VehicleType          227272 non-null object
RegistrationYear     227272 non-null int64
Gearbox              227272 non-null object
Power                227272 non-null int64
Model                227272 non-null object
Kilometer            227272 non-null int64
RegistrationMonth    227272 non-null int64
FuelType             227272 non-null object
Brand                227272 non-null object
NotRepaired          227272 non-null object
dtypes: int64(5), object(6)
memory usage: 20.8+ MB


In [21]:
df['FuelType'].value_counts()

petrol      148678
gasoline     74338
lpg           3576
cng            422
hybrid         166
other           47
electric        45
Name: FuelType, dtype: int64

Other вероятно обозначает гужевую тягу. 47 машин - удалим их.

In [22]:
df = df.query('FuelType != "other"')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227225 entries, 3 to 354367
Data columns (total 11 columns):
Price                227225 non-null int64
VehicleType          227225 non-null object
RegistrationYear     227225 non-null int64
Gearbox              227225 non-null object
Power                227225 non-null int64
Model                227225 non-null object
Kilometer            227225 non-null int64
RegistrationMonth    227225 non-null int64
FuelType             227225 non-null object
Brand                227225 non-null object
NotRepaired          227225 non-null object
dtypes: int64(5), object(6)
memory usage: 20.8+ MB


In [23]:
df['NotRepaired'].value_counts()

no     204596
yes     22629
Name: NotRepaired, dtype: int64

Вряд ли там все правдиво, но не нам об этом судить.

Переведем все признаки в категориальные при помощи labelencoder для оптимизации работы.

In [24]:
labelencoder = LabelEncoder()
df['NotRepaired'] = labelencoder.fit_transform(df['NotRepaired'])
df['VehicleType'] = labelencoder.fit_transform(df['VehicleType'])
df['Gearbox'] = labelencoder.fit_transform(df['Gearbox'])
df['Model'] = labelencoder.fit_transform(df['Model'])
df['FuelType'] = labelencoder.fit_transform(df['FuelType'])
df['Brand'] = labelencoder.fit_transform(df['Brand'])
df

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired
3,1500,5,2001,1,75,116,150000,6,5,37,0
4,3600,5,2008,1,69,101,90000,7,2,31,0
5,650,4,1995,1,102,11,150000,10,5,2,1
6,2200,1,2004,1,109,8,150000,8,5,25,0
10,2000,4,2004,1,105,10,150000,12,5,19,0
...,...,...,...,...,...,...,...,...,...,...,...
354358,1490,5,1998,1,50,143,150000,9,5,37,0
354359,7900,4,2010,1,140,116,150000,7,2,37,0
354362,3200,4,2004,1,225,140,150000,5,5,30,1
354366,1199,1,2000,0,101,106,125000,3,5,32,0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227225 entries, 3 to 354367
Data columns (total 11 columns):
Price                227225 non-null int64
VehicleType          227225 non-null int64
RegistrationYear     227225 non-null int64
Gearbox              227225 non-null int64
Power                227225 non-null int64
Model                227225 non-null int64
Kilometer            227225 non-null int64
RegistrationMonth    227225 non-null int64
FuelType             227225 non-null int64
Brand                227225 non-null int64
NotRepaired          227225 non-null int64
dtypes: int64(11)
memory usage: 20.8 MB


Данные подготовлены, можно приступать к обучению моделей.

# 2. Обучение моделей

Приступим к обучению моделей. Начнем с линейной регрессии, предварительно разделив массив на признаки и целевой признак.

In [26]:
features_train, features_valid, target_train, target_valid = train_test_split(
    df.drop('Price', axis=1), df.Price, test_size=0.25, random_state=12345)

Введем rmse.

In [27]:
def rmse(target, predictions):
    return (((target-predictions)**2).mean())**0.5 

Обучим линейную регрессию и посчитаем rmse.

In [28]:
%%time

model = LinearRegression()
model.fit(features_train, target_train)

CPU times: user 45.1 ms, sys: 8.74 ms, total: 53.8 ms
Wall time: 66.1 ms


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

68 милисекунд на обучение линейной регрессии.

In [29]:
%%time

predictions = model.predict(features_valid)
print(rmse(target_valid, predictions))

2902.289960862128
CPU times: user 17.5 ms, sys: 14.1 ms, total: 31.6 ms
Wall time: 5.73 ms


RMSE 2902 и 22 милисекунды на предсказания. 
Приступим к градиентным бустингам. Начнем с CatBoostRegressor и попробуем подобрать лучшие параметры при помощи GridSearchCV.

In [30]:
%%time

cat_features = ['VehicleType', 'Gearbox', 'Model', 'RegistrationMonth', 'FuelType', 'Brand', 'NotRepaired']

model_cat = CatBoostRegressor()
 
params = {'iterations': range (1, 130, 25),
          'depth': range (1, 10, 2),
          }
 
grid = GridSearchCV(model_cat, params, cv=5)

grid.fit(features_train, target_train)

grid.best_params_

0:	learn: 4693.0263807	total: 55.9ms	remaining: 0us
0:	learn: 4690.5475627	total: 6.9ms	remaining: 0us
0:	learn: 4693.4381647	total: 6.99ms	remaining: 0us
0:	learn: 4696.4756972	total: 6.89ms	remaining: 0us
0:	learn: 4694.5322080	total: 6.87ms	remaining: 0us
0:	learn: 4693.0263807	total: 7.41ms	remaining: 185ms
1:	learn: 4637.4990891	total: 94.3ms	remaining: 1.13s
2:	learn: 4584.6392591	total: 101ms	remaining: 772ms
3:	learn: 4534.3406545	total: 193ms	remaining: 1.06s
4:	learn: 4486.2326439	total: 200ms	remaining: 839ms
5:	learn: 4440.4917693	total: 295ms	remaining: 982ms
6:	learn: 4396.4995808	total: 302ms	remaining: 819ms
7:	learn: 4354.7014232	total: 392ms	remaining: 882ms
8:	learn: 4315.0037299	total: 399ms	remaining: 753ms
9:	learn: 4277.1338473	total: 491ms	remaining: 785ms
10:	learn: 4240.7842924	total: 498ms	remaining: 679ms
11:	learn: 4205.7427175	total: 590ms	remaining: 688ms
12:	learn: 4171.5778080	total: 596ms	remaining: 596ms
13:	learn: 4139.1744699	total: 689ms	remaining:

{'depth': 9, 'iterations': 126}

За 20 минут и 15 секунды мы определили, что лучшая комбинация 9-126. Обучим модель.

In [31]:
%%time

model_cat = CatBoostRegressor(iterations = 126, depth = 9)

model_cat.fit(features_train, target_train, cat_features=cat_features, verbose=1)

0:	learn: 4645.7872519	total: 390ms	remaining: 48.8s
1:	learn: 4544.8554085	total: 790ms	remaining: 49s
2:	learn: 4446.9715853	total: 1.1s	remaining: 45.2s
3:	learn: 4351.7657775	total: 1.5s	remaining: 45.9s
4:	learn: 4262.7566912	total: 1.89s	remaining: 45.7s
5:	learn: 4175.4042434	total: 2.28s	remaining: 45.7s
6:	learn: 4091.2394208	total: 2.59s	remaining: 44.1s
7:	learn: 4010.3105467	total: 2.98s	remaining: 44s
8:	learn: 3932.6466103	total: 3.3s	remaining: 42.9s
9:	learn: 3857.4023058	total: 3.69s	remaining: 42.8s
10:	learn: 3785.5287634	total: 4.08s	remaining: 42.7s
11:	learn: 3715.3674436	total: 4.39s	remaining: 41.7s
12:	learn: 3647.5824253	total: 4.78s	remaining: 41.6s
13:	learn: 3582.1093862	total: 5.18s	remaining: 41.4s
14:	learn: 3519.9073475	total: 5.57s	remaining: 41.2s
15:	learn: 3460.1169996	total: 5.88s	remaining: 40.4s
16:	learn: 3401.7090766	total: 6.27s	remaining: 40.2s
17:	learn: 3345.5673270	total: 6.67s	remaining: 40s
18:	learn: 3291.5569790	total: 6.98s	remaining:

47 секунд на обучение на параметрах 9-126.

In [32]:
%%time

probabilities_valid = model_cat.predict(features_valid)
print(rmse(target_valid, probabilities_valid))

1812.6027878202603
CPU times: user 196 ms, sys: 33.8 ms, total: 230 ms
Wall time: 174 ms


RMSE 1812 и 231 милисекунда на предсказания. 

In [33]:
%%time
 
gbm = lgb.LGBMRegressor()

params_lgb = {'n_estimators': range (1, 100, 10),
              'learning_rate': [0.1, 0.2]}
 
grid_lgb = GridSearchCV(gbm, params_lgb, cv=5)

grid_lgb.fit(features_train, target_train)

grid_lgb.best_params_

CPU times: user 5min 39s, sys: 0 ns, total: 5min 39s
Wall time: 5min 42s


{'learning_rate': 0.2, 'n_estimators': 91}

На lgb лучшее сочетание 0.2-91 за 10 минут 29 секунд.

In [34]:
%%time

gbm = lgb.LGBMRegressor(n_estimators = 91, learnig_rate = 0.2)

gbm.fit(features_train, target_train, eval_set=[(features_valid, target_valid)])

[1]	valid_0's l2: 1.90388e+07
[2]	valid_0's l2: 1.6454e+07
[3]	valid_0's l2: 1.43428e+07
[4]	valid_0's l2: 1.26145e+07
[5]	valid_0's l2: 1.11867e+07
[6]	valid_0's l2: 1.00134e+07
[7]	valid_0's l2: 9.04207e+06
[8]	valid_0's l2: 8.22621e+06
[9]	valid_0's l2: 7.55712e+06
[10]	valid_0's l2: 6.99519e+06
[11]	valid_0's l2: 6.52135e+06
[12]	valid_0's l2: 6.11653e+06
[13]	valid_0's l2: 5.76963e+06
[14]	valid_0's l2: 5.47616e+06
[15]	valid_0's l2: 5.21607e+06
[16]	valid_0's l2: 4.98863e+06
[17]	valid_0's l2: 4.80054e+06
[18]	valid_0's l2: 4.62214e+06
[19]	valid_0's l2: 4.47179e+06
[20]	valid_0's l2: 4.33829e+06
[21]	valid_0's l2: 4.22443e+06
[22]	valid_0's l2: 4.12258e+06
[23]	valid_0's l2: 4.0171e+06
[24]	valid_0's l2: 3.93807e+06
[25]	valid_0's l2: 3.87226e+06
[26]	valid_0's l2: 3.80953e+06
[27]	valid_0's l2: 3.7416e+06
[28]	valid_0's l2: 3.68268e+06
[29]	valid_0's l2: 3.63378e+06
[30]	valid_0's l2: 3.58759e+06
[31]	valid_0's l2: 3.54753e+06
[32]	valid_0's l2: 3.50928e+06
[33]	valid_0's l2: 3

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learnig_rate=0.2, learning_rate=0.1,
              max_depth=-1, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=91, n_jobs=-1, num_leaves=31,
              objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0)

10 секунд на обучение lgb.

In [35]:
%%time


probabilities_valid_lgb = gbm.predict(features_valid)
print(rmse(target_valid, probabilities_valid_lgb))

1684.0667352063354
CPU times: user 555 ms, sys: 0 ns, total: 555 ms
Wall time: 514 ms


RMSE 1684 и 525 милисекунды на предсказания lgb.

# 3. Анализ моделей

Итак, для предсказания стоимости автомобилей мы попробовали использовать три разных модели. 

Первая - модель линейной регрессии. 68 милисекунд на обучение и 22 милисекунды на предсказания. RMSE равно 2902.

Вторая - модель градиентного бустинга CatBoostRegressor. 20 минут 15 секунд на подбор параметров, 47 секунд на обучение, 231 милисекунда на предсказания и RMSE равен 1812.

Третья - модель градиентного бустинга LGBMRegressor. 10 минут 29 секунд на подбор параметров, 10 секунд на обучение, 588 милисекунд на предсказание и RMSE равен 1684.

Очевидно, что ключевым параметром для нас является отклонение, а время выполнения вторично. Но когда массив данных становится в разы больше того, который мы имеем сейчас, то необходимо узнать у заказчика насколько скоро ему требуется результат. 

Вынося за скобки время работы каждой модели, нашим лидером становится градиентный бустинг LGBMRegressor - он показал при помощи перебора параметров модели наименьший результат квадратичного отклонения. Соответственно это является сигналом, что мы получим наиболее точные предсказания. 

Линейная регрессия сильно отстала от бустинга CatBoostRegressor по показателю RMSE, но при этом время работы этой модели намного ниже чем у бустингов, а поэтому в условиях сжатых сроков и очень большого массива данных можно использовать линейную регрессию. Допустим, для быстрой проверки какой-либо гипотезы, для получения релевантного значения.

CatBoostRegressor в свою очередь показал себя с не самой лучшей стороны. Метрика RMSE  хуже чем у LGBM, а время выполнения в два раза больше. Поэтому сценария для пользования этим бустингом пока нет.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнена загрузка и подготовка данных
- [x]  Выполнено обучение моделей
- [x]  Есть анализ скорости работы и качества моделей